# IMPORTS AND CONSTANTS


In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import h3
import folium
import osmnx as ox
from shapely import wkt
from folium.plugins import HeatMap
from shapely.geometry import Polygon
from shapely.geometry import Point
import os

In [3]:
RESOLUTION = 8
MOSCOW = "Moscow"

# getting the small areas

In [4]:
def visualize_polygons(geometry):

    lats, lons = get_lat_lon(geometry)

    m = folium.Map(location=[sum(lats)/len(lats), sum(lons)/len(lons)], zoom_start=13, tiles='cartodbpositron')

    overlay = gpd.GeoSeries(geometry).to_json()
    folium.GeoJson(overlay, name = 'boundary').add_to(m)

    return m

# учитывая геометрию, верните широту и долготу

def get_lat_lon(geometry):

    lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
    lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)
    return lat, lon

# данные о городе МОСКВЕ могут быть извлечены из объектов с тегом : {'граница':'административный'}

cities = ["Moscow"]
ms_org = ox.geometries_from_place(cities, {'boundary':'administrative'}).reset_index()
ms_org = ms_org.dropna(subset=['name'])

In [5]:
def create_hexagons(geoJson):
    hexagons = list(h3.polyfill(geoJson, RESOLUTION))
    return hexagons

# divide Moscow into hexagons by first finding the administrative districts and then split the latter into hexagons

districts = ['Северный административный округ', "Западный административный округ", "Северо-Западный административный округ"
             , "Северо-Восточный административный округ", "Юго-Восточный административный округ", 
             "Южный административный округ", "Юго-Западный административный округ", "Восточный административный округ", 
             "Центральный административный округ", "Новомосковский административный округ"]
ms_r = ms_org[ms_org['name'].isin(districts)]

names = ms_r['name']
i = 0
print(names)


1935            Северный административный округ
1943            Западный административный округ
1986     Северо-Западный административный округ
2029    Северо-Восточный административный округ
2050       Юго-Восточный административный округ
2056               Южный административный округ
2067        Юго-Западный административный округ
2073           Восточный административный округ
2079         Центральный административный округ
2080      Новомосковский административный округ
Name: name, dtype: object


In [6]:
# obtain a geoJson object from a GEOMETRY object that can be used to model
# the geometry

def get_geoJson(geo):
    geoJson = json.loads(gpd.GeoSeries(geo).to_json())
    geoJson = geoJson['features'][0]['geometry']
    geoJson = {'type':'Polygon','coordinates': [np.column_stack((np.array(geoJson['coordinates'][0])[:, 1],
                                                        np.array(geoJson['coordinates'][0])[:, 0])).tolist()]}
    return geoJson

reg_cols = ['h3', 'lat', 'lng']

regions = pd.DataFrame(data=[], columns=reg_cols)

for n in names:
    geo = ms_r[ms_r['name'] == n]['geometry']
    geoJson= get_geoJson(geo)

    try:
        cells = create_hexagons(geoJson)
        reg_data = [[cell, h3.h3_to_geo(cell)[0], h3.h3_to_geo(cell)[1]] for cell in cells]
        reg = pd.DataFrame(data=reg_data, columns=reg_cols)
        regions = pd.concat([regions, reg], ignore_index=True)
        i += 1
    except:
        try:
            for po in geo:
                for p in po.geoms:
                    geoJson = get_geoJson(p)
                    cells = create_hexagons(geoJson)
                    reg_data = [[cell, h3.h3_to_geo(cell)[0], h3.h3_to_geo(cell)[1]] for cell in cells]
                    reg = pd.DataFrame(data=reg_data, columns=reg_cols)
                    regions = pd.concat([regions, reg], ignore_index=True)  
            i += 1                  
        except:
            print(n)
            pass
        
regions = regions.set_index('h3').sort_index()
print(regions.shape)
print(i)

(2086, 2)
10


In [7]:
regions = regions.drop_duplicates(keep=False)
print(regions.shape)

(2050, 2)


In [10]:
regions.to_excel("moscow_hex_8_jupyter.xlsx")

In [ ]:
print(regions.head())
print(regions.reset_index())

# Добавление более сложных функций

Давайте оценим пешеходный трафик (сколько человек приезжает из этого конкретного региона / зоны и направляется в нее), оценив общее количество раз, когда определенный автобусный маршрут проходит через эту зону

In [ ]:
# давайте рассмотрим реальные автобусные маршруты.

routes_loc = os.path.join('use_data/transport/bus_routes.xlsx') # https://op.mos.ru/EHDWSREST/catalog/export/get?id=1482330
routes = pd.read_excel(routes_loc)
try:
    routes.drop(0, inplace=True, axis=0)
except KeyError:
    print(routes.loc[:3, :])
    pass

routes = routes.loc[:, ['RouteNumber', 'DirectRouteTrack']]
routes = routes.rename(columns={"DirectRouteTrack": "route", "RouteNumber": "route_id"})

In [ ]:
import re
def clean_route(row):
    # удалите любой символ, который не является цифрой, пробелом, точкой или точкой с запятой
    row['route'] = re.sub(r'[^0-9,;.]+', "", row['route'])
    return row

routes = routes.apply(clean_route, axis=1)

# удаление недопустимых маршрутов

for id, row in routes.iterrows():
    if re.match((".*[0-9]+.*"), row['route']) is None:
        routes.drop(id, axis=0, inplace=True)

routes = routes.set_index('route_id').sort_index()
print(routes.head())

                                                      route
route_id                                                   
10        37.750304,55.658905;37.750247,55.658891;37.750...
10        37.968439,55.796229;37.968325,55.796316;37.968...
1001      36.863829,55.428193;36.864000,55.428103;36.864...
1002      36.863829,55.428193;36.864000,55.428103;36.864...
1008к     37.959253,55.121159;37.959583,55.121215;37.959...


In [ ]:
# сохраните окончательные маршруты в файле excel
routes_final_loc = os.path.join("use_data/transport/final_routes.xlsx")
routes.to_excel(routes_final_loc)

## adding the bus stations

In [17]:
rf_loc = os.path.join("use_data", "transport", "routes_frequency.xlsx")
rf = pd.read_excel(rf_loc)

try:
    rf.drop(0, axis=0, inplace=True)
except :
    pass
rf.head()

/home/ayhem18/Ayhem18/DEV/Data_science/Towards_Data_Science/ds_env/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,object_category_Id,global_id,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,volume_id,trip_type,object_category_Id_en,route_id_en,service_id_en,trip_id_en,trip_headsign_en,direction_id_en,block_id_en,volume_id_en,trip_type_en
1,NaN,1139512454,728,3112739,728_3112739_4_701,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1139513095,728,3112739,728_3112739_7_701,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1139513096,728,3112739,728_3112739_11_702,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1139513097,728,3112739,728_3112739_3_701,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,1139513099,728,3112739,728_3112739_5_701,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
rf = rf.loc[:, ['route_id', 'trip_id']]
rf.to_excel(os.path.join("use_data", "transport", "routes_frequency_final.xlsx")) # https://op.mos.ru/EHDWSREST/catalog/export/get?id=1476281

# группа по идентификатору маршрута: каждый маршрут будет связан со своей частотой

routes_count = pd.pivot_table(rf, index='route_id', values='trip_id', aggfunc=['count']).sort_index()
# лучшее название
routes_count.columns= ['count']
print(routes_count)

          count
route_id       
1002        641
1024        478
1027         53
1028        118
103         266
...         ...
988         127
990         349
993          66
995         134
999         634

[922 rows x 1 columns]


In [21]:
df_traffic = pd.merge(routes, routes_count, how='inner', right_index=True, left_index=True)
print(df_traffic.shape)
print(df_traffic.head())

(114, 2)
                                                      route  count
route_id                                                          
1002      36.863829,55.428193;36.864000,55.428103;36.864...    641
1024      37.563287,55.430887;37.562931,55.430958;37.562...    478
1028      37.562419,55.431071;37.562304,55.430659;37.562...    118
1030      37.561832,55.431727;37.562165,55.431825;37.562...      6
1063      37.835508,55.630339;37.835783,55.630098;37.835...    169


In [22]:
df_traffic_final = df_traffic.loc[:, ['count']]
df_traffic_final.to_excel("use_data/transport/bus_traffic.xlsx")

In [23]:
def add_one_route(df, route_str: str, count):
    # извлекать станции в каждой поездке
    stations = re.split(";", route_str)
    i = 0
    j = 0
    # извлеките координаты каждой станции
    for s in stations:
        lng, lat   = re.split(",", s)
        lat, lng = float(lat), float(lng)
        # получить адрес h3 каждой станции (с разрешением 8)
        h3_addr = h3.geo_to_h3(lat=lat, lng=lng, resolution=RESOLUTION)
    
        try:
            
            df.loc[h3_addr, 'route'] += count
            j += 1
        except KeyError:
            
            i += 1
            pass
    return i, j
    
def add_traffic(df, df_tra):
    df['route'] = 0
    i = 0
    j = 0
    for r , c in zip(df_tra['route'], df_tra['count']):
        i_1, j_1 = add_one_route(df, r, c)  
        i += i_1
        j += j_1  
    print(i, j)
add_traffic(regions, df_traffic)

52197 23474


In [24]:
regions['route'].value_counts()

0        1700
4905        5
6867        3
981         3
17088       2
         ... 
7249        1
2447        1
31148       1
6144        1
6764        1
Name: route, Length: 328, dtype: int64

In [25]:
# сохраните зоны и количество автобусов, которые проезжают мимо них за заранее определенный промежуток времени
regions = regions.rename(columns={"route": "bus_freq_count"}).sort_index()
regions.to_excel(f"data_{str(RESOLUTION)}/regions_bus_traff.xlsx")